In [1]:
# Constants
DATA_PATH   = "../Data/Processed/1_preprocessed_df.pkl"


TECH_JOBS = ['Cloud infrastructure engineer',
             'Developer, embedded applications or devices',
             'Data or business analyst',
             'System administrator',
             'Developer, front-end',
             'Scientist',
             'DevOps specialist',
             'Developer, game or graphics',
             'Academic researcher',
             'Security professional',
             'Developer, QA or test',
             'Blockchain',
             'Developer, full-stack',
             'Data scientist or machine learning specialist',
             'Developer, mobile',
             'Developer, desktop or enterprise applications',
             'Developer, back-end',
             'Database administrator',
             'Engineer, data']

CORE_COLS = ['VersionControlSystem'
             'Languages',
             'Databases',
             'Platforms',
             'WebFrameworks',
             'MiscTech',
             'ToolsTech',
             'CollabTools'
]

USEFUL_COLS= ['Employment','RemoteWork',
              'MainBranch','CodingActivities','ProfessionalTech',
              'LearnCode', 'LearnCodeOnline', 'LearnCodeCoursesCert',
              'WorkExp', 'YearsCode', 'YearsCodePro', 'EdLevel',
              'OrgSize', 'Country',
              'ConvertedCompYearly', 'Currency', 'CompTotal', 'CompFreq']

In [3]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle

from pathlib import Path

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

_______

### Functions

In [4]:
# Create a Folder named Images to save figures in.
IMAGES_PATH = Path.cwd().parent / "Images"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    """
    This functions will save the current figure shown below.
    
    Args:
        fig_id: String Containing the name of the figure.
        tight_layout: Boolean to decide whether you want a tight layout or not.
        fig_extension: String to decide the type of the figure.
        resoultion: Int to decide the resolution of the figure.
        
    Returns:
        None
    """
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    
    if tight_layout:
        plt.tight_layout()
        
    plt.savefig(path, format=fig_extension, dpi=resolution)

### Questions we need to answer